In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
from datetime import datetime
import sys, os
sys.path.append(os.path.abspath("../src/"))
from data_helper import get_days, load_config

config = load_config('../config/mimic_file.yaml')


In [7]:
MIMIC_IV_path = config['project']['mimic_parent_math']
diagnoses_icd_df = pd.read_csv(MIMIC_IV_path+'hosp/diagnoses_icd.csv.gz', index_col = 0, compression='gzip')

In [2]:
icd_map = pd.read_pickle('/data/padmalab/ecg/data/external/mimic_icd_9_icd_10_map.pickle')

In [ ]:
diagnoses_icd_df['icd_10_code'] = np.nan
# Loop assign the
with tqdm(total=diagnoses_icd_df.shape[0]) as pbar:
    for index, row in diagnoses_icd_df.iterrows():
        if row['icd_version'] == 10:
            diagnoses_icd_df.loc[index, 'icd_10_code'] = row['icd_code'][:3] + '.' + row['icd_code'][3:]
        else:
            icd_9 = row['icd_code'][:3] + '.' + row['icd_code'][3:]
            if icd_map[icd_map['icd9'] == icd_9].shape[0] > 0:
                icd_10 = icd_map[icd_map['icd9'] == icd_9].iloc[0]['icd10']
                diagnoses_icd_df.loc[index, 'icd_10_code'] = icd_10            
        pbar.update(1)

 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉        | 6028377/6364520 [1:45:37<04:22, 1279.45it/s]

In [ ]:
diagnoses_icd_df.to_csv(MIMIC_IV_path+'hosp/diagnoses_icd10.csv.gz', compression='gzip')